In [1]:
import csv
import cv2
import glob
import json
import numpy as np
import pandas as pd
import time

from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.core import Reshape
from keras.models import Sequential
from keras.optimizers import SGD
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm

print ('Loading training data...')
time_load_start = time.time()         # Returns the number of ticks after a certain event.



Using TensorFlow backend.


Loading training data...


In [3]:
# Load jpg images to get image_array.
# training_images = glob.glob('./training_data/*_cnn/*.jpg')
# image_array = np.array([cv2.imread(name, cv2.IMREAD_GRAYSCALE) for name in tqdm(training_images)], dtype=np.float64)


# Load training data .npz to get label_array, unpacking what's in the saved .npz files.
# label_array = np.zeros((1, 3), 'float')
training_data = glob.glob('training_data/*.npz')        # Finds filename matching specified path or pattern.

image_array = None
data_array = None
for single_npz in training_data:                        # single_npz == one array representing one array of saved image data and user input label for that image.
    with np.load(single_npz) as data:
        train_image_temp = data['train']                      # returns the training data image array assigned to 'train' argument created during np.savez step in 'collect_training_data.py'
        print('train data shape: ', train_image_temp.shape)
        train_labels_temp = data['train_labels']        # returns the training user input data array assigned to 'train_labels' argument created during np.savez step in 'collect_training_data.py'
        print ('Original labels shape:', train_labels_temp.shape)
    label_array = np.array([label for label in tqdm(train_labels_temp)], dtype=np.float64)
    image_array = np.array([data for data in tqdm(train_image_temp)], dtype=np.float64)
    image_array = np.reshape(image_array, (len(image_array), 120, 320, 1))
    print(label_array.shape)
    print(image_array.shape)



train data shape:  (298, 38400)
Original labels shape: (298, 4)


100%|████████████████████████████████████████████████████████████████████████████| 298/298 [00:00<00:00, 149117.47it/s]


(298, 4)
(298, 120, 320, 1)
train data shape:  (329, 38400)
Original labels shape: (329, 4)


100%|████████████████████████████████████████████████████████████████████████████| 329/329 [00:00<00:00, 164433.51it/s]


(329, 4)
(329, 120, 320, 1)
train data shape:  (317, 38400)
Original labels shape: (317, 4)


100%|████████████████████████████████████████████████████████████████████████████| 317/317 [00:00<00:00, 316871.87it/s]


(317, 4)
(317, 120, 320, 1)


In [4]:
X = image_array
y = label_array
print ('Shape of feature array: ', X.shape)
print (type(X))
print ('Shape of label array: ', y.shape)
print (type(y))

Shape of feature array:  (317, 120, 320, 1)
<class 'numpy.ndarray'>
Shape of label array:  (317, 4)
<class 'numpy.ndarray'>


In [5]:
# # Normalize with l2 (not gonna use this...)
# X = preprocessing.normalize(X, norm='l2')

# Normalize from 0 to 1
X = X / 255.
# for each in tqdm(X):
#     each/255.

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(253, 120, 320, 1)
(64, 120, 320, 1)
(253, 4)
(64, 4)


In [6]:
model = Sequential()

time_load_end = time.time()
time_load_total = time_load_end - time_load_start
print ('Total time taken to load image data:', time_load_total, 'seconds')


# Get start time of Training
time_training_start = time.time()

print( 'Training...')

# CONVOLUTION

# When using THEANO backend
# model.add(Reshape((1, 120, 320), input_shape=(120, 320)))

# # When using TENSORFLOW backend
# model.add(Reshape((120, 320, 1), input_shape=(120, 320)))

model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(120 ,320, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(30, init='uniform'))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(4, init='uniform'))
model.add(Activation('softmax'))


model.summary()



Total time taken to load image data: 68.40645217895508 seconds
Training...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 120, 320, 32)  320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 120, 320, 32)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 60, 160, 32)   0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 58, 158, 32)   9248        maxpooling2d_1[0][0]             
________________

In [9]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# # train 
# for step in range(301):
#     cost = model.train_on_batch(X_train, y_train)
#     if step %  100 ==0:
#         print('train cost ', cost)
# Fit the model
model.fit(X_train, y_train,
          nb_epoch=30,
          batch_size=100)


Epoch 1/30
253/253 [==============================] - ETA: 7s - loss: 1.3824 - acc: 0.3600 - ETA: 1s - loss: 1.3785 - acc: 0.4150 - 10s - loss: 1.3766 - acc: 0.3794    
Epoch 2/30
253/253 [==============================] - ETA: 1s - loss: 1.3579 - acc: 0.5400 - ETA: 0s - loss: 1.3517 - acc: 0.5650 - 2s - loss: 1.3471 - acc: 0.5613     
Epoch 3/30
253/253 [==============================] - ETA: 1s - loss: 1.3107 - acc: 0.4900 - ETA: 0s - loss: 1.3025 - acc: 0.4950 - 2s - loss: 1.2941 - acc: 0.5138     
Epoch 4/30
253/253 

In [10]:


# Get end time of Training
time_training_end = time.time()
time_training_total = time_training_end - time_training_start
print( '')
print( 'Total time taken to train model:', time_training_total, 'seconds')

# Evalute trained model on TEST set
print( '')
print ('Evaluation of model on test holdout set:')
score = model.evaluate(X_test, y_test, batch_size=1000)
loss = score[0]
accuracy = score[1]
print( '')
print ('Loss score: ', loss)
print ('Accuracy score: ', accuracy)

# Save model as h5
timestr = time.strftime('%Y%m%d_%H%M%S')
filename_timestr = 'nn_{}.h5'.format(timestr)
model.save('nn_h5/nn_{}.h5'.format(timestr))

# Save parameters to json file
json_string = model.to_json()
with open('./logs/nn_params_json/nn_{}.json'.format(timestr), 'w') as new_json:
    json.dump(json_string, new_json)

# Save training results to csv log
row_params = [str(training_data)[-33:-2], filename_timestr, loss, accuracy]
with open('./logs/log_conv_training.csv','a') as log:
    log_writer = csv.writer(log)
    log_writer.writerow(row_params)



Total time taken to train model: 216.7913773059845 seconds

Evaluation of model on test holdout set:
64/64 [==============================] - 1s

Loss score:  0.388826549053
Accuracy score:  0.84375
